# Packages

In [16]:
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import pywrapcp
from ortools.sat.python import cp_model

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Route Optimization\\OR-Tools'

# Overview

The goal of *packing* problems is to find the best way to **pack a set of items of given sizes into containers with fixed capacities**. A typical application is loading boxes onto delivery trucks efficiently. Often, it's not possible to pack all the items, due to the capacity constraints. In that case, the problem is to find a subset of the items with maximum total size that will fit in the containers.

There are many types of packing problems. Two of the most important are knapsack problems and bin packing.

## Knapsack problems

In the simple knapsack problem, there is a single container (a knapsack). The items have values as well as sizes, and the goal is to pack a subset of the items that has maximum total value. For the special case in which value is equal to size, the goal is to maximize the total size of the packed items.

OR-Tools provides several solvers for knapsack problems in its algorithms library. There are also more general versions of the knapsack problem. Here are a couple of examples:
- **Multidimensional knapsack problems**, in which the **items have more than one physical quantity**, such as weight and volume, and the **knapsack has a capacity for each quantity**. Here, the term dimension does not necessarily refer to the usual spatial dimensions of height, length, and width. However, some problems might involve spatial dimensions, for example, finding the optimal way to pack rectangular boxes into a rectangular storage bin.
- **Multiple knapsack problems**, in which there are multiple knapsacks, and the goal is to **maximize the total value of the packed items in all knapsacks**.

Note that you can have a multidimensional problem with a single knapsack, or a multiple knapsack problem with just one dimension.

## Bin-packing problem

One of the most well-known packing problems is bin-packing, in which there are multiple containers (called bins) of equal capacity. Unlike the multiple knapsack problem, the number of bins is not fixed. Instead, the goal is to find the smallest number of bins that will hold all the items.

Here's a simple example to illustrate the difference between the multiple knapsack problem and the bin-packing problem. Suppose a company has delivery trucks, each of which has an 18,000 pound weight capacity, and 130,000 pounds of items to deliver.
- **Multiple knapsack**: You have five trucks and you want to load a subset of the items that has maximum weight onto them.
- **Bin packing**: You have 20 trucks (more than enough to hold all the items) and you want to use the fewest trucks that will hold them all.

# Solving a Multiple Knapsack Problem

You start with a collection of items of varying weights and values. The problem is to pack a subset of the items into five bins, each of which has a maximum capacity of 100, so that the total packed value is a maximum.

## MIP solution

In [17]:
# Create data
data = {}
data['weights'] = [
    48, 30, 42, 36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36, 36
]
data['values'] = [
    10, 30, 25, 50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 30, 25
]
assert len(data['weights']) == len(data['values'])
data['num_items'] = len(data['weights'])
data['all_items'] = range(data['num_items'])

data['bin_capacities'] = [100, 100, 100, 100, 100]
data['num_bins'] = len(data['bin_capacities'])
data['all_bins'] = range(data['num_bins'])

The data includes the following:
- `weights`: A vector containing the weights of the items.
- `values`: A vector containing the values of the items.
- `capacities`: A vector containing the capacities of the bins.

In this example, all the bins have the same capacity, but that need not be true in general.

In [18]:
# Declare MIP solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Create variables
# x[i, j] = 1 if item i is packed in bin j
x = {}
for i in data['all_items']:
    for j in data['all_bins']:
        x[i, j] = solver.BoolVar(f'x_{i}_{j}')

Each `x[(i, j)]` is a 0-1 variable, where `i` is an item and `j` is a bin. In the solution, `x[(i, j)]` will be 1 if item `i` is placed in bin `j`, and 0 otherwise.

In [19]:
# Define constraints
# Each item is assigned to at most one bin
for i in data['all_items']:
    solver.Add(sum(x[i, j] for j in data['all_bins']) <= 1)

# The amount packed in each bin cannot exceed its capacity
for j in data['all_bins']:
    solver.Add(
        sum(x[i, j] * data['weights'][i] for i in data['all_items']) <= data['bin_capacities'][j]
    )

The constraints are the following:
- Each item can be placed in at most one bin. This constraint is set by requiring the sum of `x[i, j]` over all bins `j` to be less than or equal to 1.
- The total weight packed in each bin can't exceed its capacity. This constraint is set by requiring the sum of the weights of items placed in bin `j` to be less than or equal to the capacity of the bin.

In [20]:
# Maximize total value of packed items
objective = solver.Objective()
for i in data['all_items']:
    for j in data['all_bins']:
        objective.SetCoefficient(x[i, j], data['values'][i])
objective.SetMaximization()

Note that `x[i, j] * data['values'][i]` adds the value of item `i` to the objective if the item is placed in bin `j`. If `i` is not placed in any bin, its value doesn't contribute to the objective.

In [21]:
# Invoke solver
status = solver.Solve()

# Print solution
if status == pywraplp.Solver.OPTIMAL:
    print(f'Total packed value: {objective.Value()}')
    total_weight = 0
    for j in data['all_bins']:
        print(f'Bin {j}')
        bin_weight = 0
        bin_value = 0
        for i in data['all_items']:
            if x[i, j].solution_value() > 0:
                print(f"Item {i} weight: {data['weights'][i]} value: {data['values'][i]}")
                bin_weight += data['weights'][i]
                bin_value += data['values'][i]
        print(f'Packed bin weight: {bin_weight}')
        print(f'Packed bin value: {bin_value}\n')
        total_weight += bin_weight
    print(f'Total packed weight: {total_weight}')
else:
    print('The problem does not have an optimal solution.')

Total packed value: 395.0
Bin 0
Item 5 weight: 48 value: 30
Item 7 weight: 42 value: 40
Packed bin weight: 90
Packed bin value: 70

Bin 1
Item 1 weight: 30 value: 30
Item 4 weight: 36 value: 35
Item 10 weight: 30 value: 45
Packed bin weight: 96
Packed bin value: 110

Bin 2
Item 2 weight: 42 value: 25
Item 3 weight: 36 value: 50
Packed bin weight: 78
Packed bin value: 75

Bin 3
Item 8 weight: 36 value: 30
Item 9 weight: 24 value: 35
Item 13 weight: 36 value: 30
Packed bin weight: 96
Packed bin value: 95

Bin 4
Item 12 weight: 42 value: 20
Item 14 weight: 36 value: 25
Packed bin weight: 78
Packed bin value: 45

Total packed weight: 438


## CP SAT solution

In [22]:
# Declare model
model = cp_model.CpModel()

# Create data
data = {}
data['weights'] = [
    48, 30, 42, 36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36, 36
]
data['values'] = [
    10, 30, 25, 50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 30, 25
]
assert len(data['weights']) == len(data['values'])
data['num_items'] = len(data['weights'])
data['all_items'] = range(data['num_items'])

data['bin_capacities'] = [100, 100, 100, 100, 100]
data['num_bins'] = len(data['bin_capacities'])
data['all_bins'] = range(data['num_bins'])

# Create variables
# x[i, b] = 1 if item i is packed in bin b.
x = {}
for i in data['all_items']:
    for j in data['all_bins']:
        x[i, j] = model.NewBoolVar(f'x_{i}_{j}')

# Create constraints
# Each item is assigned to at most one bin
for i in data['all_items']:
    model.AddAtMostOne(x[i, j] for j in data['all_bins'])

# The amount packed in each bin cannot exceed its capacity
for j in data['all_bins']:
    model.Add(
        sum(x[i, j] * data['weights'][i] for i in data['all_items']) <= data['bin_capacities'][j]
    )

# Create objective function
objective = []
for i in data['all_items']:
    for j in data['all_bins']:
        objective.append(cp_model.LinearExpr.Term(x[i, j], data['values'][i]))
model.Maximize(cp_model.LinearExpr.Sum(objective))

# Invoke solver
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print solution
if status == cp_model.OPTIMAL:
    print(f'Total packed value: {solver.ObjectiveValue()}')
    total_weight = 0
    for j in data['all_bins']:
        print(f'Bin {j}')
        bin_weight = 0
        bin_value = 0
        for i in data['all_items']:
            if solver.Value(x[i, j]) > 0:
                print(f"Item {i} weight: {data['weights'][i]} value: {data['values'][i]}")
                bin_weight += data['weights'][i]
                bin_value += data['values'][i]
        print(f'Packed bin weight: {bin_weight}')
        print(f'Packed bin value: {bin_value}')
        total_weight += bin_weight
    print(f'Total packed value: {total_weight}')
else:
    print('The problem does not have an optimal solution.')

Total packed value: 395.0
Bin 0
Item 4 weight: 36 value: 35
Item 9 weight: 24 value: 35
Item 14 weight: 36 value: 25
Packed bin weight: 96
Packed bin value: 95
Bin 1
Item 3 weight: 36 value: 50
Item 5 weight: 48 value: 30
Packed bin weight: 84
Packed bin value: 80
Bin 2
Item 2 weight: 42 value: 25
Item 7 weight: 42 value: 40
Packed bin weight: 84
Packed bin value: 65
Bin 3
Item 8 weight: 36 value: 30
Item 12 weight: 42 value: 20
Packed bin weight: 78
Packed bin value: 50
Bin 4
Item 1 weight: 30 value: 30
Item 10 weight: 30 value: 45
Item 13 weight: 36 value: 30
Packed bin weight: 96
Packed bin value: 105
Total packed value: 438


# The Bin Packing Problem

In this example, items of various weights need to be packed into a set of bins with a common capacity. Assuming that there are enough bins to hold all the items, the problem is to find the fewest that will suffice.

In [23]:
# Create data
def create_data_model():
    """Create the data for the example."""
    data = {}
    weights = [48, 30, 19, 36, 36, 27, 42, 42, 36, 24, 30]
    data['weights'] = weights
    data['items'] = list(range(len(weights)))
    data['bins'] = data['items']
    data['bin_capacity'] = 100
    return data
data = create_data_model()

The data includes the following:
`weights`: A vector containing the weights of the items.
`bin_capacity`: A single number giving the capacity of the bins.
There are no values assigned to the items because the goal of minimizing the number of bins doesn't involve value.

Note that `num_bins` is set to the number of items. This is because if the problem has a solution, then the weight of every item must be less than or equal to the bin capacity. In that case, the maximum number of bins you could need is the number of items, because you could always put each item in a separate bin.

In [24]:
# Declare solver
solver = pywraplp.Solver.CreateSolver('SCIP')

# Create variables
# x[i, j] = 1 if item i is packed in bin j
x = {}
for i in data['items']:
    for j in data['bins']:
        x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

# y[j] = 1 if bin j is used
y = {}
for j in data['bins']:
    y[j] = solver.IntVar(0, 1, 'y[%i]' % j)

As in the multiple knapsack example, you define an array of variables `x[(i, j)]`, whose value is 1 if item `i` is placed in bin `j`, and 0 otherwise.

For bin packing, you also define an array of variables, `y[j]`, whose value is 1 if bin `j` is used — that is, if any items are packed in it — and 0 otherwise. The sum of the `y[j]` will be the number of bins used.

In [25]:
# Define constraints
# Each item must be in exactly one bin
for i in data['items']:
    solver.Add(sum(x[i, j] for j in data['bins']) == 1)

# The amount packed in each bin cannot exceed its capacity
for j in data['bins']:
    solver.Add(
        sum(x[i, j] * data['weights'][i] for i in data['items']) <= y[j] * data['bin_capacity']
    )

The constraints are the following:
- Each item must be placed in exactly one bin. This constraint is set by requiring that the sum of `x[i][j]` over all bins `j` is equal to 1. Note that this differs from the multiple knapsack problem, in which the sum is only required to be less than or equal to 1, because not all items have to be packed.
- The total weight packed in each bin can't exceed its capacity. This is the same constraint as in the multiple knapsack problem, but in this case you multiply the bin capacity on the right side of the inequalities by `y[j]`.

Why multiply by `y[j]`? Because it forces `y[j]` to equal 1 if any item is packed in bin `j`. This is so because if `y[j]` were 0, the right side of the inequality would be 0, while the bin weight on the left side would be greater than 0, violating the constraint. This connects the variables `y[j]` to the objective of the problem, for now the solver will try to minimize the number of bins for which `y[j]` is 1.

In [26]:
# Define objective
solver.Minimize(solver.Sum([y[j] for j in data['bins']]))

# Call solver and print solution
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    num_bins = 0
    for j in data['bins']:
        if y[j].solution_value() == 1:
            bin_items = []
            bin_weight = 0
            for i in data['items']:
                if x[i, j].solution_value() > 0:
                    bin_items.append(i)
                    bin_weight += data['weights'][i]
            if bin_items:
                num_bins += 1
                print('Bin number', j)
                print('   Items packed:', bin_items)
                print('   Total weight:', bin_weight)
                print()
    print()
    print('Number of bins used:', num_bins)
    print('Time =', solver.WallTime(), 'milliseconds')
else:
    print('The problem does not have an optimal solution.')

Bin number 0
   Items packed: [0, 1, 2]
   Total weight: 97

Bin number 1
   Items packed: [3, 4, 5]
   Total weight: 99

Bin number 2
   Items packed: [6, 7]
   Total weight: 84

Bin number 3
   Items packed: [8, 9, 10]
   Total weight: 90


Number of bins used: 4
Time = 121 milliseconds
